# BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Query$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $Doc$ запросу $Query$:

$$ BM25(Query, Doc) = \sum_{i=1}^{n} \text{IDF}(q_i)*\frac{TF(q_i,Doc)*(k+1)}{TF(q_i,Doc)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где    
$$$$
$\text{IDF}(q_i)$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в корпусе   
$n(q_i)$ — количество документов, содержащих слово $q_i$

>$TF(q_i,Doc)$ - частота слова $q_i$ в документе $Doc$    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75  
$l(d)$ - длина документа (количество слов в нём)   
$avgdl$ — средняя длина документа в корпусе    

In [1]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 

import time

In [2]:
texts = [
    'киса',
    'мама',
    'мыла',
    'раму',
    'киса-мама мыла раму'
]

# from sklearn.datasets import fetch_20newsgroups
# texts = fetch_20newsgroups(subset='train')['data']
# texts = texts[:5000]
# len(texts)

## run in numpy


$$ BM25(Query, Doc) = \sum_{i=1}^{n} \text{IDF}(q_i)*\frac{TF(q_i,Doc)*(k+1)}{TF(q_i,Doc)+k(1-b+b\frac{l(d)}{avgdl})} $$ 

In [9]:
# indexing in numpy
    
count_vectorizer = CountVectorizer()
tf_vectorizer = TfidfVectorizer(use_idf=False, norm='l2')
tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2')

x_count_vec = count_vectorizer.fit_transform(texts).toarray() # для индексации запроса 
x_tf_vec = tf_vectorizer.fit_transform(texts).toarray() # матрица с tf
x_tfidf_vec = tfidf_vectorizer.fit_transform(texts).toarray() # матрица для idf

idf = tfidf_vectorizer.idf_
idf = np.expand_dims(idf, axis=0)
tf = x_tfidf_vec

k = 2
b = 0.75

len_d = x_count_vec.sum(axis=1)
print(len_d)
avdl = len_d.mean()


A = idf * tf * (k + 1)

B_1 = (k * (1 - b + b * len_d / avdl))
B_1 = np.expand_dims(B_1, axis=-1) 

B = tf + B_1
matrix = A / B


[1 1 1 1 4]


In [8]:
matrix

array([[2.08387345, 0.        , 0.        , 0.        ],
       [0.        , 2.08387345, 0.        , 0.        ],
       [0.        , 0.        , 2.08387345, 0.        ],
       [0.        , 0.        , 0.        , 2.08387345],
       [0.53467806, 0.53467806, 0.53467806, 0.53467806]])

In [4]:
(k * (1 - b + b * len_d / avdl))


array([1.4375, 1.4375, 1.4375, 1.4375, 4.25  ])

In [5]:
B_1 = (k * (1 - b + b * len_d / avdl))
np.expand_dims(B_1, axis=-1) 

array([[1.4375],
       [1.4375],
       [1.4375],
       [1.4375],
       [4.25  ]])

In [6]:
x_tfidf_vec

array([[1. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. ],
       [0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 1. ],
       [0.5, 0.5, 0.5, 0.5]])

In [7]:
# run in numpy

query = 'киса'

query_count_vec = count_vectorizer.transform([query]).toarray()

# start = time.time()
np.dot(matrix, query_count_vec.T)
# print(time.time() - start)


array([[2.08387345],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.53467806]])